# Import libraries

In [ ]:
import pandas as pd
from pathlib import Path

# Set path

In [ ]:
yearmonth = "202511"

In [ ]:
base = Path("../../gc_directory").resolve()

source_dir = base / '03_grade_samples'
output_dir = base / rf"04_database/{yearmonth[:4]}/{yearmonth}"

# Retrieve CSV files

In [ ]:
gs_dir = source_dir / yearmonth[:4] / yearmonth
grade_samples = list(gs_dir.rglob("*_db.csv"))
print(len(grade_samples), 'CSV files')
grade_samples

In [ ]:
csv_files = []

for item in grade_samples:
    df = pd.read_csv(item)
    csv_files.append(df)
    break

# Merged data

In [ ]:
df_merged = pd.concat(csv_files)

df_merged["max_depth"] = df_merged["z_lh"] - df_merged["z_dp"]
df_merged['depth_from'] = 0

df_merged

# Convert to database format

## Collar

In [ ]:
df_collar = df_merged.copy()
df_collar = df_collar[["BHID", "x_lh", "y_lh", "z_lh", "max_depth", 'd4']]

df_collar.columns = ['hole_id', 'x', 'y', 'z', 'max_depth', 'date']

df_collar["date"] = df_collar["date"].astype(str)
df_collar['date'] = pd.to_datetime(df_collar['date'], format='%Y%m%d', errors='coerce')
df_collar["date"] = df_collar["date"].dt.strftime("%Y-%m-%d")

df_collar

## Survey

In [ ]:
df_survey = df_merged.copy()
df_survey = df_survey[['BHID']]

df_survey['azimuth'] = 0
df_survey['depth'] = 0
df_survey['dip'] = -90

df_survey

## Assay

In [ ]:
df_assay = df_merged.copy()
df_assay = df_assay[['BHID','Unique ID', 'depth_from', 'max_depth', "Cu_final", "Au", "Ag", "CuO_Soluble"]]

df_assay

# Export CSVs

In [ ]:
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
df_collar.to_csv(output_dir / "collar.csv", index=False)
df_survey.to_csv(output_dir / "survey.csv", index=False)
df_assay.to_csv(output_dir / "assay.csv", index=False)